# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#346efe; border-radius: 100px 100px; text-align:center">Torch inference notebook</h1></span>

<br>
<h2 style = "font-size:16px" 

This is the inference notebook made for training with  https://www.kaggle.com/vladvdv/pytorch-train-notebook-arcface-gem-pooling/notebook  
    
Modifications for version 30:
* replaced supervized KNeighborsClassifier with unsupervized NearestNeighbors   
* corrected gridsearch for determining optim "new_individual" threhsold* (there are used the same training data as the ones the model was trained, for training the NearestNeighbors algorithm, and then the same validation data that the model was trained to predict on the NearestNeighbors algorithm.  
   
To do:
* Implement all folds model blending

In [ ]:
import pickle
import os
import gc
import cv2
import math
import copy
import time
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import joblib
from tqdm import tqdm
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import albumentations as A
from albumentations.pytorch import ToTensorV2
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("../input/timm-pytorch-image-models")
import timm
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors

In [ ]:
CONFIG = {"seed": 21, # choose your lucky seed
          "img_size": 768, # training image size
          "model_name": "tf_efficientnet_b4", # training model arhitecture
          "num_classes": 15587, # total individuals in training data
          "test_batch_size": 16, # choose acording to the training arhitecture and image size 
          "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"), # gpu
          "test_mode":False, # selects just the first 2000 samples from the test data, usefull for debuging purposes
          "percentage_new_from_test":10, # how much of the test data is estimated to be "new_individual"
          "threshold":0.1, # it will be overwriten after prediction histogram           
          "neigh":100, #cnn neighbors 
          "n_fold":5, # nr of folds that the model has been trained
          # ArcFace Hyperparameters
          "s": 30.0, 
          "m": 0.30,
          "ls_eps": 0.0,
          "easy_margin": False
          }

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CONFIG['seed'])

In [ ]:
def get_test_file_path(id):
    return f"{TEST_DIR}/{id}"

def get_train_file_path(id):
    return f"{TRAIN_DIR}/{id}"

ROOT_DIR = '../input/happy-whale-and-dolphin'
TEST_DIR = '../input/happy-whale-and-dolphin/test_images'
TRAIN_DIR = '../input/happy-whale-and-dolphin/train_images'
# weights_path = "../input/dummymodel13/Loss11.1684_epoch13_f0.bin"

if CONFIG["test_mode"]==True:
    df_test = pd.read_csv(f"{ROOT_DIR}/sample_submission.csv")[:2000]
    df_train = pd.read_csv(f"{ROOT_DIR}/train.csv")[:2000]
else:
    df_test = pd.read_csv(f"{ROOT_DIR}/sample_submission.csv")
    df_train = pd.read_csv(f"{ROOT_DIR}/train.csv")  



df_test['file_path'] = df_test['image'].apply(get_test_file_path)
df_train['file_path'] = df_train['image'].apply(get_train_file_path)
train_labels = np.array(df_train['individual_id'].values)

#split into train and valid like in the training notebook for validating NearestNeighbors approach 
# trainFold = 0 # this model was trained on fold 0
# skf = StratifiedKFold(n_splits=CONFIG['n_fold'])
# for fold, ( _, val_) in enumerate(skf.split(X=df_train, y=train_labels)):
#       df_train.loc[val_ , "kfold"] = fold
# df_train_cnn = df_train[df_train.kfold != trainFold].reset_index(drop=True)
# df_valid_cnn = df_train[df_train.kfold == trainFold].reset_index(drop=True)

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df_train['individual_id'])
df_train['dummy_labels'] = le.transform(df_train['individual_id'])
df_test['dummy_labels'] = 0
n_classes = max(df_train['dummy_labels'].values)+1
print(n_classes)

# print(df_train['file_path'])

#hardcode dummy label for input in ArcMargin forward function
# df_test['dummy_labels'] = 0
# df_train_cnn['dummy_labels'] = 0
# df_valid_cnn['dummy_labels'] = 0



In [ ]:
class HappyWhaleDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['file_path'].values
        self.labels = df['dummy_labels'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path = self.file_names[index]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = self.labels[index]
        
        if self.transforms:
            img = self.transforms(image=img)["image"]
        return img_path, img, torch.tensor(label, dtype=torch.long)
        

In [ ]:
import torchvision.models as models


class ClassifierModule(nn.Module):
    def __init__(self):
        super(ClassifierModule,self).__init__()
        self.layer1 = nn.Linear(1000,n_classes)
        self.net = models.resnet18()
        for p in self.net.parameters():
            p.requires_grad=True

    def forward(self,x):
        x1 = self.net(x)
        y = self.layer1(x1)
        return y

model = ClassifierModule()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

In [ ]:
data_transforms = {
    "test": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            ),
        ToTensorV2()], p=1.)
    
}

In [ ]:

model.to(CONFIG['device']);
#predict first on train dataset to extract embeddings
train_dataset = HappyWhaleDataset(df_train, transforms=data_transforms["test"])
train_loader = DataLoader(train_dataset, batch_size=CONFIG['test_batch_size'], 
                          num_workers=4, shuffle=True, pin_memory=True)

# valid_dataset = HappyWhaleDataset(df_valid, transforms=data_transforms["test"])
# valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['test_batch_size'], 
#                           num_workers=4, shuffle=False, pin_memory=True)

test_dataset = HappyWhaleDataset(df_test, transforms=data_transforms["test"])
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'], 
                          num_workers=4, shuffle=False, pin_memory=True)

# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(params=list(model.parameters()), lr=5e-4, betas=(0.9, 0.999))
model.train()

In [ ]:
PATH = "/kaggle/working/ckpt.pt"

In [ ]:
# ONLY RUN IF CHECKPOINT NEEDS TO BE LOADED

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
model.train()
for epoch in range(40):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        _, inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
#         print(labels)
#         print(inputs.shape)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        niter_print = 500
        if i % niter_print == niter_print-1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / niter_print:.9f}')
            running_loss = 0.0
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                }, PATH)

print('Finished Training') 

In [ ]:
# Testing
sft = nn.Softmax()
model.eval()

import csv  

header = ['image', 'predictions']

sf = open('countries.csv', 'w', encoding='UTF8')
writer = csv.writer(sf)
writer.writerow(header)
    
for i, data in enumerate(test_loader, 0):
    # get the inputs; data is a list of [inputs, labels]
    path, inputs, labels = data
    inputs = inputs.cuda()
    labels = labels.cuda()

    # forward
    outputs = sft(model(inputs))
    
    tk = torch.topk(outputs, 4).indices
    tklbls = le.inverse_transform(tk.view(-1).cpu()).reshape(tk.shape)
    for idx, p in enumerate(path):
        opt = p.split('/')[-1]+","+tklbls[idx][0]+" "+tklbls[idx][1]+" "+tklbls[idx][2]+" "+tklbls[idx][3]+" new_individual"
        print(opt)
        writer.writerow([p.split('/')[-1], tklbls[idx][0]+" "+tklbls[idx][1]+" "+tklbls[idx][2]+" "+tklbls[idx][3]+" new_individual"])
sf.close()
print("DONE!")

In [ ]:
PATH = "../input/ckpt.pt"

torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            }, PATH)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])